### Importing packages and modules

In [1]:
# module for building the pyomo model
import pyomo.environ as pe
# module for solving the pyomo model
import pyomo.opt as po

### Create the model

In [2]:
model = pe.ConcreteModel() 

Order to build the model:
1. Sets
1. Parameters
1. Variables
1. Objective function
1. Constraints

#### Sets

$f$: food type {'feed', 'forage'}

$n$: nutrient {'protein', 'calcium','vitamin'}

In [3]:
model.food_type = pe.Set(initialize=['feed', 'forage'])
model.nutrient = pe.Set(initialize=['protein', 'calcium','vitamin'])

#### Parameters

$Content_{f,n}$: nutrient n content in type of food f [g/kg]

In [4]:
nutrient_content = {
    ('feed','protein'): 30,
    ('forage','protein'): 45,
    ('feed','calcium'): 2,
    ('forage','calcium'): 1,
    ('feed','vitamin'): 0.01,
    ('forage','vitamin'): 0.005
    }

model.nutrient_content = pe.Param(model.food_type, model.nutrient, initialize = nutrient_content)

$Need_n$: daily nutrient n need [g]

In [5]:
nutrient_need = {
    'protein': 700,
    'calcium': 28,
    'vitamin': 0.15,
    }

model.nutrient_need = pe.Param(model.nutrient, initialize = nutrient_need)

$Cost_f$: cost per type of food f [€/kg]

In [6]:
food_cost = {'feed': 0.3, 'forage': 0.35}

model.food_cost = pe.Param (model.food_type, initialize = food_cost)

#### Variables

$x_f$: kg of food $f$ in daily diet [kg]

In [7]:
model.food_kg = pe.Var(model.food_type, within = pe.NonNegativeReals)

#### Objective Function

min $\sum_f Cost_f*x_f$

In [8]:
def obj_rule(model):
    return sum((model.food_cost[f] * model.food_kg[f]) for f in model.food_type)

model.cost = pe.Objective(rule = obj_rule, sense = pe.minimize)

#### Constraints

Constraint #1: minimum daily need of nutrient n in diet

$\sum_f Content_{f,n}*x_f >= Need_n \quad \forall n$

In [9]:
model.eq_nutrient_need = pe.ConstraintList()
for n in model.nutrient:
    model.eq_nutrient_need.add(
        sum(model.nutrient_content[f,n]*model.food_kg[f] for f in model.food_type) >= model.nutrient_need[n]
    )

#### Solver definition and solve statement

In [10]:
solver = po.SolverFactory('gurobi')
results = solver.solve (model, tee=True)

Set parameter Username
Set parameter LicenseID to value 2704768
Academic license - for non-commercial use only - expires 2026-09-08
Read LP format model from file /var/folders/pr/nst7vndn0l10vqp0s73wclhr0000gn/T/tmp9ada5gv1.pyomo.lp
Reading time = 0.00 seconds
x1: 3 rows, 2 columns, 6 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x8d2d0f5e
Coefficient statistics:
  Matrix range     [5e-03, 4e+01]
  Objective range  [3e-01, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 7e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.259000e+02   0.000000e+00      0s
       2    6.1666667e+00   0.000000e+00   0.0000

In [11]:
print(round(pe.value(model.cost),2))

6.17


In [12]:
for f in model.food_type:
    print('{} kg of {} in diet'.format(round(pe.value(model.food_kg[f]),2), pe.value(f)))

10.83 kg of feed in diet
8.33 kg of forage in diet


In [13]:
for n in model.nutrient:
    total_nutrient=sum(pe.value(model.nutrient_content[f,n])*pe.value(model.food_kg[f]) for f in model.food_type)
    if n == 'vitamin':
        total_nutrient = 1000*total_nutrient
        metric = 'mg'
    else:
        metric = 'g'
    print('{} {} of {} in diet'.format(total_nutrient, metric, n))

700.0 g of protein in diet
30.0 g of calcium in diet
150.0 mg of vitamin in diet
